# OpenAI function calling notebook

Docs: https://platform.openai.com/docs/guides/function-calling
More examples: https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

Basics of function calling:
* The function calling feature basically compels the model to output a JSON object containing arguments to call one or many functions. It does not actually call the function. 
* By adjusting the `tool_choice` arg, you can force the model to always call one or more functions, always call a specific function, or force the model only to generate a user facing message.

Basic steps for function calling:
1. Call the model with user query and a set of functions in the `functions` parameter
2. The model can choose to call one or more functions and will output a stringified JSON object. The model may hallucinate parameters.
3. Parse the JSON and call your function with the provided arguments, if they exist. 
4. Can call the model again appending the function response and let the model summarize the results.

Let's write a tool (function) that generates Opentrons Python code based on a natural language command.

In [30]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [32]:
default_deck_state = {
    "pipettes": {
        "left": {"type": "p300_single", "tip_racks": ["1"]},
        "right": {"type": "p20_multi", "tip_racks": ["4", "5"]}
    },
    "modules": {
        "1": "temperature module",
        "3": "magnetic module"
    },
    "labware": {
        "1": "opentrons_96_tiprack_300ul",
        "2": "corning_96_wellplate_360ul_flat",
        "3": "nest_96_wellplate_100ul_pcr_full_skirt",
        "4": "opentrons_96_tiprack_20ul",
        "5": "opentrons_96_tiprack_20ul",
        "6": "nest_12_reservoir_15ml"
    }
}

In [20]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "generate_opentrons_code",
            "description": "Generate Python code for Opentrons OT-2 robot based on a natural language command. Use the default deck state unless the contents of the command suggest a different deck state, in which case override the defaults.",
            "parameters": {
                "type": "object",
                "properties": {
                    "command": {
                        "type": "string",
                        "description": "The natural language command to be translated into Opentrons Python code"
                    }
                },
                "required": ["command"]
            }
        }
    }
]

In [36]:
def generate_opentrons_code(command):
    # Always use the default_deck_state
    deck_state = default_deck_state
    
    # Here, you would implement the logic to generate the code
    # using the command and the deck_state
    
    generated_code = f"""
# Generated Opentrons code for: {command}
# Using default deck state: {json.dumps(deck_state, indent=2)}

from opentrons import protocol_api

metadata = {{'apiLevel': '2.13'}}

def run(protocol: protocol_api.ProtocolContext):
    # TODO: Implement the steps to execute the command
    # This would involve translating the natural language command
    # into specific Opentrons API calls, using the default deck state
    pass
"""
    return generated_code

available_functions = {
    "generate_opentrons_code": generate_opentrons_code
}

In [37]:
messages = [
    {"role": "system", "content": "You are an AI assistant that helps translate natural language commands into Opentrons Python code. Always use the provided default deck state unless the user explicitly specifies changes to it."},
    {"role": "system", "content": f"Default deck state: {default_deck_state}"},
    {"role": "user", "content": "I want to transfer 100 µL from well A1 to well B1 on a 96-well plate in slot 2."}
]

In [38]:

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

In [39]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uTWaZ9YqOb9YGPAMsq0H61v9', function=Function(arguments='{"command":"transfer 100 µL from well A1 to well B1 on a 96-well plate in slot 2"}', name='generate_opentrons_code'), type='function')])

Let's break down the initial response message from the model. 
* `content=None`: This means the assistant didn't generate any text content and instead decided to call a function.
* `function_call=None`: This is depreciated and has been replaced by `tool_calls`
* `tool_calls=[...]`: This is the key part. Inside, we have a `ChatCompletionMessageToolCall` object the a unique `id`, a `function` entry with details about the function being called, and `type=function` for the type of tool call. 
* The `Function` object contains the name of the function (`name='generate_opentrons_code'`) and `arguments={"command": "..."}` – a JSON string containing arguments for the function. These args can be hallucinated.

In [40]:
tool_calls = response_message.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_uTWaZ9YqOb9YGPAMsq0H61v9', function=Function(arguments='{"command":"transfer 100 µL from well A1 to well B1 on a 96-well plate in slot 2"}', name='generate_opentrons_code'), type='function')]

The `response_message.tool_calls` returns `None` if no tools were called.

In [41]:
if tool_calls:
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        if function_name in available_functions:
            function_response = available_functions[function_name](**function_args)
            print(f"Generated Opentrons Code:\n{function_response}")
            
            messages.append({
                "role": "function",
                "name": function_name,
                "content": function_response
            })
    
    # Get a new response from the model to explain the generated code
    second_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    
    print("\nExplanation of the generated code:")
    print(second_response.choices[0].message.content)
else:
    print(response_message.content)

Generated Opentrons Code:

# Generated Opentrons code for: transfer 100 µL from well A1 to well B1 on a 96-well plate in slot 2
# Using default deck state: {
  "pipettes": {
    "left": {
      "type": "p300_single",
      "tip_racks": [
        "1"
      ]
    },
    "right": {
      "type": "p20_multi",
      "tip_racks": [
        "4",
        "5"
      ]
    }
  },
  "modules": {
    "1": "temperature module",
    "3": "magnetic module"
  },
  "labware": {
    "1": "opentrons_96_tiprack_300ul",
    "2": "corning_96_wellplate_360ul_flat",
    "3": "nest_96_wellplate_100ul_pcr_full_skirt",
    "4": "opentrons_96_tiprack_20ul",
    "5": "opentrons_96_tiprack_20ul",
    "6": "nest_12_reservoir_15ml"
  }
}

from opentrons import protocol_api

metadata = {'apiLevel': '2.13'}

def run(protocol: protocol_api.ProtocolContext):
    # TODO: Implement the steps to execute the command
    # This would involve translating the natural language command
    # into specific Opentrons API calls, usin